# *Retrieval Augmented Generation* con las conferencias mañaneras usando LangChain

## Descargando los datos de mi otro repositorio

In [ ]:
import tempfile
from pathlib import Path
import subprocess

temporary_directory = tempfile.mkdtemp()

In [ ]:

conferencias_repo_url = "https://github.com/fferegrino/mananeras.git"
conferencias_repo_dir = Path(temporary_directory, "mananeras")

subprocess.run(["git", "clone", "-q", "--single-branch",
                "--branch", 'cf-llm-1',
                "--depth", "1", conferencias_repo_url, str(conferencias_repo_dir)])

## Cargando los documentos

Todo sistema *RAG* comienza cargando un conjunto inicial de documentos. 

In [ ]:
import mananeras

conferencias = mananeras.lee_todas(conferencias_repo_dir)

In [ ]:
len(conferencias)

In [ ]:
print(conferencias[1].titulo)
print(conferencias[1].fecha)
print(conferencias[1].participaciones[6])

In [ ]:
dialogos_presidente = []

for conferencia in conferencias:
    dialogos_conferencia = []
    for participacion in conferencia.participaciones:
        hablante = participacion.hablante.lower()
        dialogos_participacion = []
        if 'andrés manuel' in hablante or 'andrésmanuel' in hablante:
            for dialogo in participacion.dialogos:
                dialogos_participacion.append(dialogo)
        if len(dialogos_participacion) > 0:
            dialogos_conferencia.append("\n".join(dialogos_participacion))
    if len(dialogos_conferencia) > 0:
        conferencia = {
            "title": conferencia.titulo,
            "document": "\n".join(dialogos_conferencia)
        }
        dialogos_presidente.append(conferencia)

In [ ]:
len(dialogos_presidente)

In [ ]:
print(dialogos_presidente[0]['document'][:500])

## Load documents using a document loader

Since our text is already in memory, we need to create a custom `DocumentLoader`:

In [ ]:
from typing import AsyncIterator, Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document

class DialogosPresidenteLoader(BaseLoader):

    def __init__(self, conferencias):
        self.conferencias = conferencias

    def lazy_load(self) -> Iterator[Document]:

        for conferencia in self.conferencias:
            dialogos_conferencia = []
            for participacion in conferencia.participaciones:
                hablante = participacion.hablante.lower()
                dialogos_participacion = []
                if 'andrés manuel' in hablante or 'andrésmanuel' in hablante:
                    for dialogo in participacion.dialogos:
                        dialogos_participacion.append(dialogo)
                if len(dialogos_participacion) > 0:
                    dialogos_conferencia.append("\n".join(dialogos_participacion))
            if len(dialogos_conferencia) > 0:
                metadata = {
                    "title": conferencia.titulo,
                    "date": conferencia.fecha
                }

                yield Document(
                    page_content="\n".join(dialogos_conferencia),
                    metadata=metadata
                )

In [ ]:
dialogos_presidente_loader = DialogosPresidenteLoader(conferencias)

for document in dialogos_presidente_loader.lazy_load():
    print(document.metadata['title'])
    print(document.metadata['date'])
    print(document.page_content[:500])
    break

In [ ]:
docs = dialogos_presidente_loader.load()

## Divide documento en partes (*chunks*)

Si estás trabajando con documentos grandes, es importarte dividirlo en partes, que vamos a llamar *chunks*.

Esto cumple dos funciones:

 * Mejorar la relevancia semántica de nuestros embeddings: un documento muy grande puede cubrir demasiados temas, mientras que uno pequeño puede estar más enfocado en un solo tópico
 * Facilitar la tarea del modelo de lenuaje generativo – *chunks* más pequeños hacen que la ventana de contexto sea más pequeña

El proceso de división tiene varios parámetros: el tamaño del *chunk* y el tamaño de traslape entre *chunks*.

Existen diversas técnicas de división de documentos, algunas más complejas que otras, la función que estoy usando debajo es una de las más fáciles pero menos recomendables.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, chunk_overlap=10, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
print(len(all_splits))
print(all_splits[0].page_content)

## Calculando embeddings & almacenándolos en la BD (Chroma)

Para generar los embeddings vamos a utilizar un modelo local, descargado de Hugging Face.

In [ ]:
from pathlib import Path
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata

embedding_model = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-mpnet-base-v2")

vector_store = "./vector_store"

if Path(vector_store).exists():
    vector_store_loaded = Chroma(persist_directory=vector_store, embedding_function=embedding_model)
else:
    vector_store = Chroma.from_documents(
        documents=filter_complex_metadata(all_splits),
        embedding=embedding_model,
        persist_directory="./vector_store"
    )

In [ ]:
embedding = embedding_model.embed_query("Hola mundo")   

In [ ]:
len(embedding)

## Ejecutando queries

In [ ]:
vector_store_loaded = Chroma(persist_directory="vector_store", embedding_function=embedding_model)

In [ ]:
retriever = vector_store_loaded.as_retriever(search_type="mmr", search_kwargs={"k": 6})

In [ ]:
pregunta = "¿Qué significa ser aspiracionista?"

In [ ]:
retrieved_docs = retriever.invoke(pregunta)

for doc in retrieved_docs:
    print(doc.metadata['title'])
    # print(doc.metadata['date'])
    print(doc.page_content[:500])
    print()

## Usando una LLM para generar respuestas

In [ ]:
from langchain.prompts import ChatPromptTemplate

rag_prompt_template = ChatPromptTemplate.from_template("""Eres Andrés Manuel Lopez Obrador, presidente de México.
Responde a la pregunta basándote en el contexto de lo dicho por el presidente.
El contexto está delimitado por las comillas invertidas.
Contesta como si la respuesta la estuviera dando Andrés Manuel Lopez Obrador.

```
{context}
```

Pregunta: {question}
""")

In [ ]:
rag_prompt_template.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("¿Qué significa ser aspiracionista?"):
    print(chunk, end="", flush=True)

In [ ]:
from openai import OpenAI

client = OpenAI()

def query_llm(prompt, model="gpt-3.5-turbo"):
    completions = client.chat.completions.create(

    model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )

    return completions.choices[0].message.content

print(pregunta)
print()
prompt = """Eres Andrés Manuel Lopez Obrador, presidente de México.
Responde a la pregunta como si la respuesta la estuviera dando Andrés Manuel Lopez Obrador.

Pregunta: {question}
"""
final_prompt = prompt.format(question=pregunta)
print(query_llm(final_prompt))